In [1]:
from google.auth import default
credentials, project = default()

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.config_loader import config
from src.translation import review_translations_with_anthropic, review_translated_phrases_batch
from src.utils import get_first_n_items
from src.gcs_storage import get_translated_phrases_path, read_from_gcs, upload_to_gcs
print(config.TARGET_LANGUAGE_NAME)
COLLECTION = "WarmUp150"

Swedish


# Translation Refinement

## Issues with Google Translate

It can sound stiff / formal and be more related to written text rather than verbal.

## Approach

Use Sonnet 3.5 via the Anthropic API, using a tool to return adjusted translations in JSON format for re-uploading to our phrase translation store (json file)

In [18]:
# get current translated JSON file
translated_file_path =get_translated_phrases_path(collection=COLLECTION)

phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=translated_file_path)

In [19]:
test_dict = get_first_n_items(phrase_translations, 5)

In [20]:
improved_translations = review_translated_phrases_batch(test_dict)

Processing batch 1 with 5 phrases...


In [ ]:
len(improved_translations)

In [ ]:
upload_to_gcs(obj=improved_translations, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)

In [21]:
improved_translations

{'i_cant_figure_it_out': {'english': "I can't figure it out",
  'swedish': 'Jag kan inte komma på det',
  'modified': True},
 'do_you_know_the_way': {'english': 'Do you know the way?',
  'swedish': 'Vet du vägen?',
  'modified': False},
 'lets_have_tea_this_afternoon': {'english': "Let's have tea this afternoon",
  'swedish': 'Ska vi dricka te i eftermiddag?',
  'modified': True},
 'im_trying_to_lose_weight': {'english': "I'm trying to lose weight",
  'swedish': 'Jag försöker gå ner i vikt',
  'modified': False},
 'i_might_get_a_new_job': {'english': 'I might get a new job',
  'swedish': 'Jag kanske får ett nytt jobb',
  'modified': False}}